In [1]:
!pip install -q keras_metrics

In [2]:
from pathlib import Path
import glob
import random
import cv2
from numpy.random import seed
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from scipy import ndimage
from skimage import exposure
import skimage
from skimage import io
from skimage import transform as tm
import seaborn as sns
import tensorflow as tf
# from tensorflow.keras.utils import np_utils
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense,Flatten,Activation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Activation, Convolution2D, Dropout, Conv2D
from tensorflow.keras.layers import AveragePooling2D, BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Input,GaussianNoise
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import SeparableConv2D
from keras import layers
from tensorflow.keras.regularizers import l2
import keras_metrics
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from tensorflow.keras.applications.xception import Xception,preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import Adam,RMSprop,SGD
from sklearn.metrics import confusion_matrix,precision_score,recall_score
from sklearn.metrics import roc_auc_score
from keras import backend as K
%matplotlib inline

In [3]:
# for consistemt results across multiple executions
seed(3)
tf.random.set_seed(3)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!pip install unrar
!unrar x '/content/drive/MyDrive/DIP/HW2/ALL_IDB2.rar' '/content/drive/MyDrive/DIP/HW2'


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal

Enter password (will not be echoed) for /content/drive/MyDrive/DIP/HW2/ALL_IDB2.rar: 


Extracting from /content/drive/MyDrive/DIP/HW2/ALL_IDB2.rar

Creating    /content/drive/MyDrive/DIP/HW2/ALL_IDB2                   OK
Creating    /content/drive/MyDrive/DIP/HW2/ALL_IDB2/img               OK
Extracting  /content/drive/MyDrive/DIP/HW2/ALL_IDB2/img/Im001_1.tif        0%  OK 
Extracting  /content/drive/MyDrive/DIP/HW2/ALL_IDB2/img/Im002_1.tif        0%  OK 
Extracting  /content/drive/MyDrive/DIP/HW2/ALL_IDB2/img/Im003_1.tif        1%  OK 
Extracting  /content/drive/MyDrive/DIP/HW2/ALL_IDB2/img/Im004_1.tif        1%  OK 
Extracting  /content/drive/MyDrive/DIP/HW2/ALL_IDB2/img/Im005_1.tif        1%  2%  OK 
Extracting  /content/drive/MyDrive/DIP/HW2/ALL_IDB2/img/Im006_1.tif        2%  OK 
Extracting  /content/drive/MyDrive/DIP/HW2/ALL_IDB2/img/Im007_1.tif      

In [25]:
images_dir = Path('/content/drive/MyDrive/DIP/HW2/ALL_IDB2/img')
images = images_dir.glob("*.tif")

train_data = []

for img in images:
  img_address = str(img)
  partition = img_address.rpartition('.') # ('/content/drive/MyDrive/DIP/HW2/ALL_IDB2/img/Im260_0', '.', 'tif')
  if partition[0][-1] == "1":             # label of image
    train_data.append((img,1))    
  else:
    train_data.append((img,0))

print(f'len of train_data : {len(train_data)}')    
train_data = pd.DataFrame(train_data,columns=['image','label'],index = None)
train_data = train_data.sample(frac=1.).reset_index(drop=True)

len of train_data : 260


In [26]:
train_data.head()

,image,label
0,/content/drive/MyDrive/DIP/HW2/ALL_IDB2/img/Im...,1
1,/content/drive/MyDrive/DIP/HW2/ALL_IDB2/img/Im...,1
2,/content/drive/MyDrive/DIP/HW2/ALL_IDB2/img/Im...,0
3,/content/drive/MyDrive/DIP/HW2/ALL_IDB2/img/Im...,1
4,/content/drive/MyDrive/DIP/HW2/ALL_IDB2/img/Im...,1
